# Rotations

This gives an introduction to some of the functionality in REBOUND for typically encountered rotations in celestial mechanics.

REBOUND has a general `Rotation` class. Let's make one that rotates counterclockwise by 45 degrees around the z axis [0,0,1]

In [1]:
import rebound
import numpy as np

rot = rebound.Rotation(angle=np.radians(45), axis=[0,0,1])

A rotation can act on various objects. For example, we can act on any vector:

In [2]:
result = rot*[1,1,1]
result

[0.0, 1.4142135623730951, 1.0]

We can also get the inverse of any rotation object:

In [3]:
rot.inverse()*result

[1.0, 1.0, 1.0]

We can combine rotations. Here we first rotate around z axis by 90 degrees, then around the x axis by 90 degrees (order matters!)

In [4]:
r1 = rebound.Rotation(angle=np.radians(90), axis=[0,0,1])
r2 = rebound.Rotation(angle=np.radians(90), axis=[1,0,0])
r2*r1*[1,1,1]

[-0.9999999999999996, -0.9999999999999998, 1.0000000000000004]

# Useful Rotations

The `Rotation` class offers alternate constructors for various useful rotations. Let's make a simplified Solar System:

In [5]:
sim = rebound.Simulation()
date = "2023-01-01 00:00"
sim.add('Sun')
sim.add('Jupiter')
sim.add('Saturn', hash='Saturn')
sim.move_to_com()
ps = sim.particles

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Searching NASA Horizons for 'Saturn'... 
Found: Saturn Barycenter (6) (chosen from query 'Saturn')


Let's construct a rotation from our reference axes (the HORIZONS queries we're using default to the ecliptic as the reference plane, see Horizons.ipynb) to reference axes aligned with Saturn's orbit (where the new z direction is along the orbit normal, and x direction is toward pericenter).

In [6]:
rot = rebound.Rotation.to_orbital(Omega=ps['Saturn'].Omega, inc=ps['Saturn'].inc, omega=ps['Saturn'].omega)
rot*ps['Saturn'].xyz

[-5.757103045199742, -7.966390021506616, 1.199040866595169e-14]

When we act our rotation on Saturn's xyz position (in our original coordinate system, in AU), we see we get a vector with vanishing z component (good since Saturn should be in its own orbital plane!), and that Saturn is a bit past apocenter (both x and y are negative).

We could also use the inverse of this rotation, e.g., to easily get the direction toward Saturn's pericenter in our ecliptic coordinate system. In the axes aligned with Saturn's orbit, pericenter is in the x ([1,0,0]) direction, so in the ecliptic coordinate system we have

In [7]:
rot.inverse()*[1,0,0]

[-0.033243160362501856, 0.9993183063134097, -0.016056617203241773]

Now say we realize that the ecliptic plane should have very little to do with the dynamics of Saturn and Jupiter, and we want to rotate into the invariable plane, where the z direction points along the total angular momentum. We can do:

In [8]:
rot = rebound.Rotation.to_new_axes(newz=sim.calculate_angular_momentum())

We could also have passed a `newx` vector perpendicular to newz in order to specify the new x direction. If we don't, it defaults sensibly to the line of nodes at the intersection between our reference plane (here the ecliptic) and our new reference plane (perpendicular to newz, here the invariable plane)--specifically the z cross newz direction.

We can now, e.g., get Saturn's position (or any other vector) in our new coordinate system:

In [10]:
rot*ps['Saturn'].xyz

[-7.550940414537042, -6.291937086342272, -0.04931536707994988]

However, we might also want to rotate our entire Simulation into this new coordinate system, so the z axis is always a physically meaningful direction. We can do that simply with:

In [11]:
print(sim.calculate_angular_momentum())
sim.rotate(rot)
print(sim.calculate_angular_momentum())

[8.371972018903151e-05, 2.435788542866555e-05, 0.0030550236872849933]
[-8.470329472543003e-22, -2.710505431213761e-20, 0.0030562676630170155]


We see that before rotating our Simulation, the angular momentum was almost, but not quite along the z direction (the ecliptic is of course close to the invariable plane!), but after the rotation, the x and y components are at the level of the machine precision.

# Technical Detail: Copies vs in-place rotations

As a general rule, `Rotation` * `object` always returns a copy. For example, we can also act a `Rotation` on a `Particle`:

In [14]:
ecliptic_saturn = rot.inverse() * sim.particles['Saturn']
ecliptic_saturn.x, sim.particles['Saturn'].x

(2.9905677930888617, 8.147064178851359)

our rotated vector and our particle yield different x positions (because the rotation didn't affect the `sim.particles['Saturn']` we used as input). When we call the `rotate` method on a rebound object, the object is updated accordingly. For example, if we wanted to update Saturn with a rotated position (and velocity) in our Simulation, we could do:

In [15]:
sim.particles['Saturn'].rotate(rot.inverse())
ecliptic_saturn.x, sim.particles['Saturn'].x

(2.9905677930888617, 2.9905677930888617)

Now we see that the two yield the same x value, since we've actually updated the positions of the particle in our simulation. 

In most use cases, we probably want to rotate a Simulation in place with `sim.rotate(rot)`. If we do `rot*sim` we get back a shallow copy that doesn't keep any of our function pointers (see `sim.copy()`).